In [4]:
%matplotlib inline

input = open('D:/ZidLab/SolexaDataAnalysis/eIF5aD/wtcomb_Sorted.txt','r')
output = open('D:/ZidLab/SolexaDataAnalysis/eIF5aD/wtcomb.csv','w')
output.write('Name'+ ',' + 'Orient' + ',' + 'Start' + ',' + 'End' + ',' + 'Length' + ',' + 'RelativePosition' + ',' + 'Counts' + '\n')

while True:
    line = input.readline()
    if not line:
         break
    
    string = line.rstrip() 
    cells = string.split('\t')
    name = cells[0][:7] 
    orient = cells[0][6:7]
    start = cells[1]
    end = cells[2]
    length = abs(int(start) - int(end))
    riboCount = cells[4]
    if orient == 'W':
        posit = (int(cells[3])+1 - (int(start)))
        if posit > 0 and posit<(length-15):
                output.write(str(name) + ',' + str(orient) + ',' + str(start) + ',' + str(end) + ',' + str(length-15) + ',' + str(posit) + ',' + str(riboCount) + '\n')
    elif orient == "C":
        posit = ((int(start))-(int(cells[3])-1))
        if posit > 0 and posit<(length-15):
                output.write(str(name) + ',' + str(orient) + ',' + str(start) + ',' + str(end) + ',' + str(length-15) + ',' + str(posit) + ',' + str(riboCount) + '\n')
input.close()
output.close()

In [5]:
import sys
import xlrd
import pandas as pd
import numpy as np
import os
import fnmatch 
import matplotlib.pyplot as plt
input = open('D:/ZidLab/SolexaDataAnalysis/eIF5aD/wtcomb.csv','r')
output = open('D:/ZidLab/SolexaDataAnalysis/eIF5aD/wtcomb_polarity.csv','w')


%matplotlib inline
def calculatePolarity(i, l, densityati, density):
    normDist = float((2*(i+1) - (l + 1))/float(l - 1))
    polarity = densityati*normDist/density
    return polarity


df1 = pd.read_csv(input)
#names = inputpath.split('.')
Namelist = df1['Name'].tolist()

Countslist = df1['Counts'].tolist()
Orientlist = df1['Orient'].tolist()

startp = 0
for i in range (len(Namelist) - 1):
    if Namelist[i + 1] == Namelist[i]:
        Countslist[i + 1] += Countslist[i]
        
    else:
        for j in range (startp, i+1):
            Countslist[j] = Countslist[i]
        startp = i + 1
i+=1        
for j in range (startp, i+1):
    Countslist[j] = Countslist[i]
startp = i + 1 

df = pd.DataFrame({'col':Countslist})
df1['TotalCounts'] = Countslist
Totalcountlist = df1['TotalCounts'].tolist()

Countsperilist = df1['Counts'].tolist()
Lengthlist = df1['Length'].tolist()
Positlist = df1['RelativePosition'].tolist()
Polaritylist = []
Genelist = []
for i in range (len(Namelist)):
    
    polarityofi = float(calculatePolarity(Positlist[i], Lengthlist[i], Countsperilist[i], Totalcountlist[i]))
    Polaritylist.append(polarityofi)
    newname = str(Namelist[i])
    Genelist.append(newname)
    

df = pd.DataFrame({'col':Polaritylist})
df1['Polarity'] = Polaritylist

#calculate total polarity
Gene_Polarity = {}
geneP = df1['Polarity'].tolist()
for i in range (len(Namelist) - 1):
    if Namelist[i + 1] == Namelist[i]:
        geneP[i + 1] += geneP[i]
        geneP[i] = -1000
    else:
        Gene_Polarity[Namelist[i]] = geneP[i]
df = pd.DataFrame({'col':geneP})
df1['PolarityPerGene'] = geneP
d = {'Name':Genelist, 'Totalcount': Totalcountlist, 'PolarityPerGene': geneP}
dfcsv = pd.DataFrame(data=d)
dfcsv = dfcsv.replace(-1000, np.nan)
dfcsv = dfcsv.dropna(how='any',axis=0)
    

dfcsv.to_csv(output)

input.close()
output.close()